In [9]:
from torch.utils.data import random_split
from datasets import load_dataset

from utils import Config
from tokenizer import create_tokenizer, load_tokenizer

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
config = Config('config.yaml')
config.__dict__

{'DATASOURCE': 'opus_books',
 'LANG_SRC': 'ru',
 'LANG_TGT': 'it',
 'PATH_TO_MODEL_WEIGHTS': 'weights',
 'PATH_TO_EXPERIMENTS': 'runs',
 'PATH_TO_TOKENIZERS': 'tokenizers',
 'TR_FRAC': 0.95,
 'MAX_LEN': 250,
 'BATCH_SIZE': 32,
 'NUM_EPOCHS': 10,
 'EMBED_SIZE': 512,
 'HIDDEN_SIZE': 2048,
 'DROPOUT': 0.1,
 'HEADS': 8,
 'LAYERS': 6}

In [11]:
data_raw = load_dataset(
    path=config.DATASOURCE, 
    name=f"{config.LANG_TGT}-{config.LANG_SRC}", # ru -> it
    split="train"
)
data_raw['translation'][300:305]

[{'it': 'La seduta doveva durare fino alle due, senza interruzione; alle due, intervallo e colazione.',
  'ru': 'До двух часов занятия должны были идти не прерываясь, а в два часа -- перерыв и завтрак.'},
 {'it': 'Non erano ancora le due quando la grande porta a vetri dell’aula si aprì improvvisamente e qualcuno entrò.',
  'ru': 'Еще не было двух часов, когда большие стеклянные двери залы присутствия вдруг отворились, и кто-то вошел.'},
 {'it': 'Tutti i membri ritratti sotto il ritratto dell’imperatore e al di là dello specchio a tre facce, lieti della distrazione, si voltarono a guardare verso la porta; ma l’usciere che stava all’ingresso respinse subito colui che s’era infilato e richiuse la porta a vetri.',
  'ru': 'Все члены из-под портрета и из-за зерцала, обрадовавшись развлечению, оглянулись на дверь; но сторож, стоявший у двери, тотчас же изгнал вошедшего и затворил за ним стеклянную дверь.'},
 {'it': 'Quando tutto il rapporto fu letto, Stepan Arkad’ic si alzò stiracchiandosi e

In [10]:
src_tokenizer = create_tokenizer(dataset=data_raw, config=config, lang=config.LANG_SRC)
tgt_tokenizer = create_tokenizer(dataset=data_raw, config=config, lang=config.LANG_TGT)

output = src_tokenizer.encode("Привет, как у тебя дела?")
print(output.tokens)

output = tgt_tokenizer.encode("Ciao, come va?")
print(output.tokens)

['[UNK]', ',', 'как', 'у', 'тебя', 'дела', '?']
['[UNK]', ',', 'come', 'va', '?']


In [11]:
tr_size = int(0.9 * len(data_raw))
va_size = len(data_raw) - tr_size
tr_data_raw, va_data_raw = random_split(data_raw, [tr_size, va_size])
len(tr_data_raw), len(va_data_raw)

(16115, 1791)

In [12]:
#  Find the maximum length of each sentence in the source and target sentence
max_len_src = 0
max_len_tgt = 0

for item in data_raw:
    src_ids = src_tokenizer.encode(item["translation"][config.LANG_SRC]).ids
    tgt_ids = tgt_tokenizer.encode(item["translation"][config.LANG_TGT]).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

print(f"Max length of source ({config.LANG_SRC}) sentence: {max_len_src}")
print(f"Max length of target ({config.LANG_TGT}) sentence: {max_len_tgt}")

Max length of source (ru) sentence: 187
Max length of target (it) sentence: 220
